In [2]:
from keras.models import load_model
import numpy as np, csv, matplotlib.pyplot as plt, random
from keras import backend as K

TESTING_COUNT = 10
WORDS_COUNT = 133778
EPOCHS = 3

/home/aryan/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
print("Loading model")

MODEL_PATH = '{0} [{1}].hd5'.format(WORDS_COUNT, EPOCHS)
model = load_model(MODEL_PATH)
print("Done!")

Loading model
Done!


In [4]:
def get_vectors(kfunc, getting):
    print("Getting {0} from file".format(getting))
    items = {}
    ids = []
    with open("{0}.{1}.csv".format(str(WORDS_COUNT), getting)) as fd:
        reader = csv.reader(fd)
        for i, w in reader:
            items[i] = w
            ids.append([i])
    ids = np.asarray(ids)
    print("Done!")
    print("Getting vectors")
    temp = kfunc([ids])[0]
    for i, v in zip(ids, temp):
        i, v = i[0], (v[0] if getting == 'words' else v)
        items[i] = (items[i], v)
    print("Done!")
    return items


def get_words_vectors():
    kfunc = K.function(
        [model.layers[1].input],
        [model.layers[3].get_output_at(0)]
    )

    return get_vectors(kfunc, 'words')

def get_rels_vectors():
    kfunc = K.function(
        [model.layers[0].input],
        [model.layers[4].output]
    )
    
    return get_vectors(kfunc, 'rels')
    

In [16]:
words_vectors = get_words_vectors()
rels_vectors = get_rels_vectors()

Getting words from file
Done!
Getting vectors
Done!
Getting rels from file
Done!
Getting vectors
Done!


### Relaciones

In [ ]:
temp = (('0', '1'),('2', '3'), ('4', '5'), ('6', '7'))
for r in rels_vectors:
    x = rels_vectors[str(r)][1]
    s = np.linalg.norm(x-x.T)**2/(np.linalg.norm(x)*np.linalg.norm(x.T))
    print(rels_vectors[r][0],s)
print('------')
for k1,k2 in temp:
    _k1 = rels_vectors[k1][1]
    _k2 = rels_vectors[k2][1]
    t = np.linalg.norm(_k1-_k2.T)**2/(np.linalg.norm(_k1)*np.linalg.norm(_k2.T))
    print(rels_vectors[k1][0], t)
    
print('---------')
for r in rels_vectors:
    for r2 in [i for i in rels_vectors if i != r]:
        _k1 = rels_vectors[r][1]
        _k2 = rels_vectors[r2][1]
        t = np.linalg.norm(_k1-_k2.T)**2/(np.linalg.norm(_k1)*np.linalg.norm(_k2.T))


In [6]:
data = np.genfromtxt(str(WORDS_COUNT) + '.set', dtype=('<U2'))
data_rel = {}

for d in [t for t in data if t[3] == "1"]:
    if d[1] not in data_rel:
        data_rel[d[1]] = []
    data_rel[d[1]].append(tuple(d))

In [21]:
#Probar sinonimos
def choice(ss, cc):
    k = np.random.choice(len(cc))
    while cc[k] in ss:
        k = np.random.choice(len(cc))
    return cc[k]

count = 10
samples = []
while count > 0:
    temp = data_rel['0']
    elem = choice(samples, temp)
    syns = []
    for t in temp:
        if t[0] == elem[0]:
            syns.append(t)
    elem2 = choice(samples, syns)
    samples.append(elem)
    samples.append(elem2)
    count-=1

print(samples)

[('10', '0', '56', '1'), ('10', '0', '87', '1'), ('32', '0', '18', '1'), ('32', '0', '94', '1'), ('38', '0', '21', '1'), ('38', '0', '65', '1'), ('89', '0', '37', '1'), ('89', '0', '22', '1'), ('33', '0', '95', '1'), ('33', '0', '61', '1'), ('40', '0', '44', '1'), ('40', '0', '11', '1'), ('34', '0', '24', '1'), ('34', '0', '34', '1'), ('64', '0', '76', '1'), ('64', '0', '64', '1'), ('11', '0', '50', '1'), ('11', '0', '12', '1'), ('51', '0', '51', '1'), ('51', '0', '18', '1')]


In [39]:
samples = np.asarray(samples)
W1 = [[s] for s in samples[::2,2]]
W2 = [[s] for s in samples[1::2,2]]

W1 = np.asarray(W1)
W2 = np.asarray(W2)



In [50]:
pr = [[]]*len(R)
for rel in rels_vectors:
    R = np.asarray([[rel]]*len(W1))
    result = model.predict([W1,W2,R])
    for i,r in enumerate(result):
        print(p[i])
        pr[i].append((rels_vectors[rel][0], r[0]))
print(pr)

('syn', 0.9896459)
('syn', 0.8981629)
('syn', 0.43174455)
('syn', 0.94639426)
('syn', 0.577123)
('syn', 0.9225689)
('syn', 0.09305249)
('syn', 0.019649914)
('syn', 0.43766922)
('syn', 0.52761775)
('syn', 0.9896459)
('syn', 0.8981629)
('syn', 0.43174455)
('syn', 0.94639426)
('syn', 0.577123)
('syn', 0.9225689)
('syn', 0.09305249)
('syn', 0.019649914)
('syn', 0.43766922)
('syn', 0.52761775)
('syn', 0.9896459)
('syn', 0.8981629)
('syn', 0.43174455)
('syn', 0.94639426)
('syn', 0.577123)
('syn', 0.9225689)
('syn', 0.09305249)
('syn', 0.019649914)
('syn', 0.43766922)
('syn', 0.52761775)
('syn', 0.9896459)
('syn', 0.8981629)
('syn', 0.43174455)
('syn', 0.94639426)
('syn', 0.577123)
('syn', 0.9225689)
('syn', 0.09305249)
('syn', 0.019649914)
('syn', 0.43766922)
('syn', 0.52761775)
('syn', 0.9896459)
('syn', 0.8981629)
('syn', 0.43174455)
('syn', 0.94639426)
('syn', 0.577123)
('syn', 0.9225689)
('syn', 0.09305249)
('syn', 0.019649914)
('syn', 0.43766922)
('syn', 0.52761775)
('syn', 0.9896459)
(

In [41]:
result

array([[0.9896459 ],
       [0.8981629 ],
       [0.43174455],
       [0.94639426],
       [0.577123  ],
       [0.9225689 ],
       [0.09305249],
       [0.01964991],
       [0.43766922],
       [0.52761775]], dtype=float32)